In [17]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from tqdm import tqdm_notebook as tqdm
from bs4 import BeautifulSoup as bs
import requests as req

In [18]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\user\AppData\Local\Temp\ipykernel_35728\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


input을 사용해서 검색어를 입력할수있도록하고

format을 사용해서 검색어를 받아 오도로 할수 있다는거 까지만 알려드릴께요^^

In [19]:
driver.get('https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EC%B9%9C%ED%99%98%EA%B2%BD%20%EC%A0%9C%ED%92%88&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=46&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start=1')

##### 뉴스페이지 url 가져오기

In [20]:
soup = bs(driver.page_source,'html.parser')

In [21]:
search = soup.select('a.news_tit')

In [22]:
for i in search:
    print(i['href'])

https://biz.chosun.com/it-science/ict/2023/02/13/X5PF6KP5LZEVBEVOSGQR2I67OI/?utm_source=naver&utm_medium=original&utm_campaign=biz
http://news.bizwatch.co.kr/article/industry/2023/02/13/0011
http://www.sportsseoul.com/news/read/1198627?ref=naver
http://www.newsis.com/view/?id=NISX20230212_0002189611&cID=13001&pID=13000
https://www.nocutnews.co.kr/news/5894072
http://moneys.mt.co.kr/news/mwView.php?no=2023021014465934746
https://www.seoul.co.kr/news/newsView.php?id=20230213500111&wlog_tag3=naver
https://www.etoday.co.kr/news/view/2221215
https://www.news1.kr/articles/4950509
http://www.edaily.co.kr/news/newspath.asp?newsid=01692486635510192


In [23]:
# search1 = soup.find_all('a', class_='news_tit')
# for i in search1:
#     print('https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EA%B7%80%EB%86%8D+%EB%B6%88%ED%8E%B8%ED%95%9C+%EC%A0%90' + i['href'])

In [45]:
url_list = []
date_list= []
for i in range(1,10):
    page = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EC%B9%9C%ED%99%98%EA%B2%BD%20%EC%A0%9C%ED%92%88&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=46&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start={}1'.format(i-1)
    driver.get(page)
    time.sleep(1.5)

    soup = bs(driver.page_source,'html.parser')
    search = soup.select('a.news_tit')
    search1 = soup.select('div.info_group > span.info')
    for i in search:
        url_list.append(i['href'])
    for i in search1:
        date_list.append(i)
len(url_list), len(date_list)

(90, 97)

In [ ]:
# hrefs_list = []
# for i in range(1,10):
#     page = 'https://news.sbs.co.kr/news/newsflash.do?plink=NEW&cooper=SBSNEWSMAIN&pageIdx={}'.format(i)
#     hrefs = soup.select('a.news')        
#     for i in hrefs:
#         hrefs_list.append('https://news.sbs.co.kr/' + i['href'])
# len(hrefs_list)

In [ ]:
url_list

['https://biz.chosun.com/it-science/ict/2023/02/13/X5PF6KP5LZEVBEVOSGQR2I67OI/?utm_source=naver&utm_medium=original&utm_campaign=biz',
 'http://news.bizwatch.co.kr/article/industry/2023/02/13/0011',
 'http://www.sportsseoul.com/news/read/1198627?ref=naver',
 'http://www.newsis.com/view/?id=NISX20230212_0002189611&cID=13001&pID=13000',
 'https://www.nocutnews.co.kr/news/5894072',
 'http://www.getnews.co.kr/news/articleView.html?idxno=616404',
 'http://moneys.mt.co.kr/news/mwView.php?no=2023021014465934746',
 'https://www.ddaily.co.kr/news/article.html?no=257256',
 'http://www.enewstoday.co.kr/news/articleView.html?idxno=1637541',
 'https://www.seoul.co.kr/news/newsView.php?id=20230213500111&wlog_tag3=naver',
 'https://www.etoday.co.kr/news/view/2221215',
 'https://www.news1.kr/articles/4950509',
 'http://www.newsis.com/view/?id=NISX20230204_0002181085&cID=10433&pID=13000',
 'https://www.econovill.com/news/articleView.html?idxno=602370',
 'http://moneys.mt.co.kr/news/mwView.php?no=202302

In [ ]:
newsdata = {'참조주소':url_list, '날짜':date_list} 
current_news = pd.DataFrame(newsdata)
current_news

In [ ]:
# 수집 후 url list 중복 확인
real_list = set(url_list)
blog_url_list = list(real_list)
len(blog_url_list)

2368

In [ ]:
# 상세 페이지 기사 제목 & 내용
title_list = []
contents_list = []

for url in tqdm(url_list):
    driver.get(url)
    time.sleep(1)
    soup = bs(driver.page_source,'html.parser')
    
    try:
        title = soup.select('div.w_article_title > h3')
        contents = soup.select('div.text_area')
    
    except:
        title = soup.select('div.article > h1')
        contents = soup.select('div.content_area')
    
    for i in title:
        title_list.append(i.text)
        
    for i in contents:
        contents_list.append(i.text) 
len(title_list), len(contents_list)

C:\Users\user\AppData\Local\Temp\ipykernel_35728\351153141.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for url in tqdm(url_list):


  0%|          | 0/3000 [00:00<?, ?it/s]

TimeoutException: Message: timeout: Timed out receiving message from renderer: -769.244
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00E36643]
	(No symbol) [0x00DCBE21]
	(No symbol) [0x00CCDA9D]
	(No symbol) [0x00CBF55A]
	(No symbol) [0x00CBF2D8]
	(No symbol) [0x00CBDC68]
	(No symbol) [0x00CBE647]
	(No symbol) [0x00CC8568]
	(No symbol) [0x00CD4956]
	(No symbol) [0x00CD81C6]
	(No symbol) [0x00CBE9F1]
	(No symbol) [0x00CD46D5]
	(No symbol) [0x00D37057]
	(No symbol) [0x00D1FB76]
	(No symbol) [0x00CF49C1]
	(No symbol) [0x00CF5E5D]
	GetHandleVerifier [0x010AA142+2497106]
	GetHandleVerifier [0x010D85D3+2686691]
	GetHandleVerifier [0x010DBB9C+2700460]
	GetHandleVerifier [0x00EE3B10+635936]
	(No symbol) [0x00DD4A1F]
	(No symbol) [0x00DDA418]
	(No symbol) [0x00DDA505]
	(No symbol) [0x00DE508B]
	BaseThreadInitThunk [0x76167D69+25]
	RtlInitializeExceptionChain [0x7744BB9B+107]
	RtlClearBits [0x7744BB1F+191]


In [ ]:
newsdata = {'제목':title_list,'내용':contents_list} 
current_news = pd.DataFrame(newsdata)
current_news

In [ ]:
newsdata = {'제목':title_list,'날짜':date_list,'내용':contents_list,'참조주소':url_list} 
current_news = pd.DataFrame(newsdata)
current_news